# Streaming Results

Here is an example pattern if you want to stream your multiple results. Note that this is not supported for Hugging Face text completions at this time.


In [ ]:
# Note: if using a Poetry virtual environment, do not run this cell
%pip install semantic-kernel==1.1.0

In [ ]:
# Make sure paths are correct for the imports

import os
import sys

notebook_dir = os.path.abspath("")
parent_dir = os.path.dirname(notebook_dir)
grandparent_dir = os.path.dirname(parent_dir)


sys.path.append(grandparent_dir)

We will load our settings and get the LLM service to use for the notebook.

In [ ]:
from dotenv import load_dotenv
from services import Service

load_dotenv()

global_llm_service = os.getenv("GLOBAL_LLM_SERVICE")
if global_llm_service is None:
    global_llm_service = "AzureOpenAI"

selectedService = Service(global_llm_service.lower())

print(f"Using service type: {selectedService}")

Let's load our settings and validate that the required ones exist.

In [ ]:
if selectedService == Service.AzureOpenAI:
    api_key = os.getenv("AZURE_OPENAI_API_KEY")
    chat_deployment_name = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
    text_deployment_name = os.getenv("AZURE_OPENAI_TEXT_DEPLOYMENT_NAME")
    endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    base_url = os.getenv("AZURE_OPENAI_BASE_URL")
    api_version = os.getenv("AZURE_OPENAI_API_VERSION")

    assert api_key is not None, "Azure OpenAI API key is not set"
    assert chat_deployment_name is not None, "Azure OpenAI Chat deployment name is not set"
    assert text_deployment_name is not None, "Azure OpenAI Text deployment name is not set"
elif selectedService == Service.OpenAI:
    api_key = os.getenv("OPENAI_API_KEY")
    chat_model_id = os.getenv("OPENAI_CHAT_MODEL_ID")
    text_model_id = os.getenv("OPENAI_TEXT_MODEL_ID")
    org_id = os.getenv("OPENAI_ORG_ID")

    assert api_key is not None, "OpenAI API key is not set"
    assert chat_model_id is not None, "OpenAI Chat model ID is not set"
    assert text_model_id is not None, "OpenAI Text model ID is not set"

First, we will set up the text and chat services we will be submitting prompts to.


In [ ]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    AzureChatPromptExecutionSettings,  # noqa: F401
    AzureTextCompletion,
    OpenAIChatCompletion,
    OpenAIChatPromptExecutionSettings,  # noqa: F401
    OpenAITextCompletion,
    OpenAITextPromptExecutionSettings,  # noqa: F401
)
from semantic_kernel.contents import ChatHistory  # noqa: F401

kernel = Kernel()

service_id = None
if selectedService == Service.OpenAI:
    from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

    service_id = "default"
    oai_chat_service = OpenAIChatCompletion(
        service_id="oai_chat",
        ai_model_id=chat_model_id,
        org_id=org_id,
        api_key=api_key,
    )
    oai_text_service = OpenAITextCompletion(
        service_id="oai_text",
        ai_model_id=text_model_id,
        org_id=org_id,
        api_key=api_key,
    )
elif selectedService == Service.AzureOpenAI:
    from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

    service_id = "default"
    aoai_chat_service = AzureChatCompletion(
        service_id="aoai_chat",
        api_key=api_key,
        deployment_name=chat_deployment_name,
        endpoint=endpoint,
        base_url=base_url,
        api_version=api_version,
    )
    aoai_text_service = AzureTextCompletion(
        service_id="aoai_text",
        api_key=api_key,
        deployment_name=text_deployment_name,
        endpoint=endpoint,
        base_url=base_url,
        api_version=api_version,
    )

# Configure Hugging Face service
if selectedService == Service.HuggingFace:
    from semantic_kernel.connectors.ai.hugging_face import (
        HuggingFacePromptExecutionSettings,  # noqa: F401
        HuggingFaceTextCompletion,
    )

    hf_text_service = HuggingFaceTextCompletion(ai_model_id="distilgpt2", task="text-generation")

Next, we'll set up the completion request settings for text completion services.


In [ ]:
oai_prompt_execution_settings = OpenAITextPromptExecutionSettings(
    service_id="oai_text",
    max_tokens=150,
    temperature=0.7,
    top_p=1,
    frequency_penalty=0.5,
    presence_penalty=0.5,
)

## Streaming Open AI Text Completion


In [ ]:
if selectedService == Service.OpenAI:
    prompt = "What is the purpose of a rubber duck?"
    stream = oai_text_service.get_streaming_text_contents(prompt=prompt, settings=oai_prompt_execution_settings)
    async for message in stream:
        print(str(message[0]), end="")  # end = "" to avoid newlines

## Streaming Azure Open AI Text Completion


In [ ]:
if selectedService == Service.AzureOpenAI:
    prompt = "provide me a list of possible meanings for the acronym 'ORLD'"
    stream = aoai_text_service.get_streaming_text_contents(prompt=prompt, settings=oai_prompt_execution_settings)
    async for message in stream:
        print(str(message[0]), end="")

## Streaming Hugging Face Text Completion


In [ ]:
if selectedService == Service.HuggingFace:
    hf_prompt_execution_settings = HuggingFacePromptExecutionSettings(
        service_id="hf_text",
        extension_data={
            "max_new_tokens": 80,
            "top_p": 1,
            "eos_token_id": 11,
            "pad_token_id": 0,
        },
    )

In [ ]:
if selectedService == Service.HuggingFace:
    prompt = "The purpose of a rubber duck is"
    stream = hf_text_service.get_streaming_text_contents(
        prompt=prompt, prompt_execution_settings=hf_prompt_execution_settings
    )
    async for text in stream:
        print(str(text[0]), end="")  # end = "" to avoid newlines

Here, we're setting up the settings for Chat completions.


In [ ]:
oai_chat_prompt_execution_settings = OpenAIChatPromptExecutionSettings(
    service_id="oai_chat",
    max_tokens=150,
    temperature=0.7,
    top_p=1,
    frequency_penalty=0.5,
    presence_penalty=0.5,
)

## Streaming OpenAI Chat Completion


In [ ]:
if selectedService == Service.OpenAI:
    content = "You are an AI assistant that helps people find information."
    chat = ChatHistory()
    chat.add_system_message(content)
    stream = oai_chat_service.get_streaming_chat_message_contents(
        chat_history=chat, settings=oai_chat_prompt_execution_settings
    )
    async for text in stream:
        print(str(text[0]), end="")  # end = "" to avoid newlines

## Streaming Azure OpenAI Chat Completion


In [ ]:
az_oai_chat_prompt_execution_settings = AzureChatPromptExecutionSettings(
    service_id="aoai_chat",
    max_tokens=150,
    temperature=0.7,
    top_p=1,
    frequency_penalty=0.5,
    presence_penalty=0.5,
)

In [ ]:
if selectedService == Service.AzureOpenAI:
    content = "You are an AI assistant that helps people find information."
    chat = ChatHistory()
    chat.add_system_message(content)
    chat.add_user_message("What is the purpose of a rubber duck?")
    stream = aoai_chat_service.get_streaming_chat_message_contents(
        chat_history=chat, settings=az_oai_chat_prompt_execution_settings
    )
    async for text in stream:
        print(str(text[0]), end="")  # end = "" to avoid newlines